In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
from astropy.time import Time
from scipy.interpolate import interp1d
import math
import copy
import glob
import subprocess
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

In [2]:
#some useful functions

# Read all files for a given instrument, resample and concatenate
def read_single_meteo_file(filename,keep=['Date', 'pressure', 'Temperature', 'Humidity', 'windSpeed', 'windDir'],set_index=True):
    df = pd.read_csv(filename, header=1, parse_dates={'Date':['YYYY/MM/JJ',' hh:mn:sec(HL)']})
    # Remove spaces in column names
    df.rename(str.strip,axis='columns',inplace=True)
    # Rename some columns
    df.rename(columns={"outTemp": "Temperature", "outHumidity": "Humidity"},inplace=True)
    if (keep != 'all'):
        columns=keep
        index_to_remove = df.columns.drop(keep)
        df = df.drop(columns=index_to_remove)
    if (set_index):
        # Use Date as index
        df.set_index('Date',inplace=True)
    return (df)

def read_single_pml_file(filename,keep=['Date', 'Seeing', 'Cn2_ground', 'Cn2_150', 'Cn2_250'],set_index=True):
    df = pd.read_csv(filename,header=22,parse_dates={'Datetmp':['Date','Time_UT']})
    df.rename(columns={"Datetmp": "Date", "Tau0(ms)": "Tau0", "Seeing[arcsec]": "Seeing", "Isop[arcsec]": "Isop"},inplace=True)
    if (keep != 'all'):
        index_to_remove = df.columns.drop(keep)
        df = df.drop(columns=index_to_remove)
    if (set_index):
        df.set_index('Date',inplace=True)
    return(df)

def read_single_gdimm_file(filename,keep=['Seeing', 'Date', 'Isop', 'Tau0'],set_index=True):
    df = pd.read_csv(filename,header=20,parse_dates={'Datetmp':['Date','Time_UT']})
    df.rename(columns={"Datetmp": "Date", "Tau0(ms)": "Tau0"},inplace=True)
    df.insert(16, 'Seeing', 0.5*(df['epsT']+df['epsL']))

    if (keep != 'all'):
        index_to_remove = df.columns.drop(keep)
        df  = df.drop(columns=index_to_remove)
    if (set_index):
        df.set_index('Date',inplace=True)
    return (df)

###############################################################################

# Routine to prepare training samples from single contiguous dataframe
def split_series(series, n_past, n_future, out_column_index=5):
    #
    # n_past ==> no of past observations
    #
    # n_future ==> no of future observations 
    #
    out_column_index
    X, y = list(), list()
    for window_start in range(len(series)):
        past_end = window_start + n_past
        future_end = past_end + n_future
        if future_end > len(series):
            break
        # slicing the past and future parts of the window
        past, future = series[window_start:past_end, :], series[past_end:future_end, [out_column_index]]
        X.append(past)
        y.append(future)
    return np.array(X), np.array(y)  


# Routine to fill missing values with random values from data distribution
def fill_missing_values(dataframe,random=False):

    if (random):
        columns = dataframe.columns
        for col in columns:
            mean = dataframe[col].mean()
            std  = dataframe[col].std() 
            mask_nans = dataframe[col].isnull().values
            randoms = np.random.standard_normal(mask_nans.sum())*std + mean
            dataframe.loc[mask_nans,col]=randoms
    else:
        dataframe.interpolate(method='spline',order=3,inplace=True)
    return(dataframe)

In [3]:
def simple_data_preparation(data_dir = '/Users/cgiordano/Documents/Travail/WRF/Calern_ML/Data', 
                     meteo_dir='CATS',meteo_tag='meteo_cats_*.csv',
                     gdimm_dir='GDIMM_tmp',gdimm_tag='new_r0Alt_2*.csv',
                     pml_dir='PBL_tmp',pml_tag='new_Cn2_2*.csv',
                     sampling_rate_min=5,interpolate=True,day_only=True):

    sampling_rate_str = '%dmin'%sampling_rate_min
    

    GDIMM_files = sorted(glob.glob(data_dir+'/'+gdimm_dir+'/'+gdimm_tag))
    nGDIMM = len(GDIMM_files)
    PML_files = sorted(glob.glob(data_dir+'/'+pml_dir+'/'+pml_tag))
    nPML = len(PML_files)
    METEO_files = np.array(glob.glob(data_dir+'/'+meteo_dir+'/'+meteo_tag))
    filetest = np.array(['-'.join(item.split('_')[-1].split('.')[0].split('-')[::-1]) for item in METEO_files])
    idx = np.argsort(filetest)
    METEO_files = [item for item in METEO_files[idx]]
    nMETEO = len(METEO_files)
    # return (gdimm_files,pml_files,meteo_files)
    

    framelist=[]
    for file in GDIMM_files:
        #print("Processing file %s"%file)
        df = read_single_gdimm_file(file)
        if (len(df)==0):
            continue
        df = df.resample(sampling_rate_str).bfill()
        if (interpolate):
            df.interpolate('spline',order=1,inplace=True)
        framelist.append (df)
    gdimm_data = pd.concat(framelist)
    print('gdimm_data',gdimm_data)
 


    framelist=[]
    for file in PML_files:
        #print("Processing file %s"%file)
        df = read_single_pml_file(file)
        if (len(df)==0):
            continue
        df = df.resample(sampling_rate_str).bfill()
        if (interpolate):
            df.interpolate('spline',order=1,inplace=True)
        framelist.append(df)
    pml_data = pd.concat(framelist)
    print('pml_data',pml_data)
   

    framelist = []
    for file in METEO_files:
        #print("Processing file %s"%file)
        df = read_single_meteo_file(file)
        if (len(df)==0):                  
            continue
        df = df.resample(sampling_rate_str).bfill()
        if (interpolate):
            df.interpolate('spline',order=1,inplace=True)
        framelist.append(df)
    meteo_data = pd.concat(framelist)
    print('meteo_data',meteo_data)
    

    
    # Now do some merging of data. First outer join of pml and gdimm, and take (nan)mean of seing values as final seeing value
    # This will allow to have seeing measurements during days and nights. It's ok because overlapping measurements are broadly compatible

    if not day_only:

        gdimm_pml = pd.merge(pml_data,gdimm_data,left_index=True,right_index=True,how='outer',suffixes=['_pml','_gdimm'])
        seeing_gdimm = gdimm_pml['Seeing_gdimm'].values
        seeing_pml   = gdimm_pml['Seeing_pml'].values
        seeing = np.nanmean(np.vstack((seeing_gdimm,seeing_pml)),axis=0)
        gdimm_pml['Seeing']=seeing
        # Get rid of per instrument seeing measurements, just keep mean seeing value
        gdimm_pml.drop(columns=['Seeing_gdimm','Seeing_pml'])
        to_merge = gdimm_pml
    else:
        to_merge = pml_data # Do not include night only gdimm data
    
    print('to_merge',to_merge)

    # Now (inner) merge with meteo data
    final = pd.merge(meteo_data,to_merge,left_index=True,right_index=True)
    
    print('final',final)
    
    # OK, some final filtering
    final[final.values<0]=np.nan
    final.Seeing[final.Seeing.values>5]=np.nan
    # Now check if we have the day_only filter
    # Drop some columns
    if not day_only:
        final.drop(columns=['Cn2_ground','Cn2_150','Cn2_250','Isop','Tau0'],inplace=True)
    else:
        final.drop(columns=['Cn2_ground','Cn2_150','Cn2_250'],inplace=True)

    # Add group index, based on time jumps bigger than 300s (this needs to be in sync with sampling_rate !!)
    final['groups'] = (final.index.to_series().diff().dt.seconds > 300).cumsum()

    
    print('final',final)
    
    return final

In [4]:
final = simple_data_preparation(data_dir = r'C:\Users\Mary-\Desktop\PhD\First year\ML LSTM\forecast-main\test_data', 
                     meteo_dir='meteo',meteo_tag='meteo_cats_*.csv',
                     gdimm_dir='gdimm',gdimm_tag='new_r0Alt_2*.csv',
                     pml_dir='pml',pml_tag='new_Cn2_2*.csv',
                     sampling_rate_min=5,interpolate=True,day_only=True)

gdimm_data                      Seeing  Isop   Tau0
Date                                    
2018-06-02 02:35:00   0.940  1.47  -1.00
2018-06-02 02:40:00   0.780  1.72  -1.00
2018-06-03 21:50:00   1.070  1.37   3.48
2018-06-03 21:55:00   1.385  1.47   2.70
2018-06-03 22:00:00   0.950  1.43   3.31
...                     ...   ...    ...
2018-06-30 02:20:00   0.575  1.45   4.83
2018-06-30 02:25:00   0.480  1.69   5.46
2018-06-30 02:30:00   0.450  1.89  30.80
2018-06-30 02:35:00   0.490  1.55   6.30
2018-06-30 02:40:00   0.495  1.57   7.38

[901 rows x 3 columns]
pml_data                      Seeing    Cn2_ground       Cn2_150       Cn2_250
Date                                                                 
2018-06-05 11:30:00  3.3793  2.216900e-14  1.602000e-14  3.025600e-16
2018-06-05 11:35:00  2.0115  1.442700e-14  4.339900e-16  1.465600e-15
2018-06-05 11:40:00  2.0115  1.442700e-14  4.339900e-16  1.465600e-15
2018-06-05 11:45:00  2.0115  1.442700e-14  4.339900e-16  1.465600e-15
201

In [19]:
def prepare_learning_sets(dataframe,input_sequence_length_min=60,
                          output_sequence_length_min=30,test_size=0.2,
                          sampling_rate_min=5, 
                          out_column='Temperature',
                          return_scaler = True):

    '''
    This routine first splits the dataframe into training and testing, scales all columns, and finally arrange the
    training and testing sets into their final form for training and testing. The final form consists in an array 
    of time series of input_sequence_length_min minutes and corresponding target time series of 
    output_sequence_length_min minutes. 
    '''
    n_input_sequence = input_sequence_length_min//sampling_rate_min
    n_output_sequence = output_sequence_length_min//sampling_rate_min
    n_full_sequence  = n_input_sequence+n_output_sequence
    n_features = dataframe.shape[1]
    X_train = np.empty((0,n_input_sequence,n_features))
    X_test = np.empty((0,n_input_sequence,n_features))
    Y_train = np.empty((0,n_output_sequence,1))
    Y_test = np.empty((0,n_output_sequence,1))
    out_column_index = dataframe.columns.get_loc(out_column)
    print('n_full_sequence',n_full_sequence)

    for group in np.unique(dataframe['groups']):
        current = dataframe[dataframe.groups==group]  #split the groups
        

        # Split and check that current dataframes have enough samples
        train_df, test_df = train_test_split(current,test_size=test_size,shuffle=False)
        if (train_df.shape[0] < n_full_sequence or test_df.shape[0] < n_full_sequence):
            #print("train_df",train_df.shape[0])
            continue
            
        
        
        for col in train_df.columns:
            scaler = MinMaxScaler(feature_range=(-1,1))
            ss = scaler.fit_transform(train_df[col].values.reshape((-1,1)))
            
            ss = ss.reshape(len(ss)) # Get rid of extra dim
            train_df.loc[:,col] = ss
            train_df.rename(columns={col:"scaled_%s"%col},inplace=True)
            if col==out_column:
                out_scaler = scaler
            # Now process the corresponding column from the test dataframe
            ss = scaler.transform(test_df[col].values.reshape((-1,1)))
            ss = ss.reshape(len(ss))
            test_df.loc[:,col] = ss
            test_df.rename(columns={col:"scaled_%s"%col},inplace=True)

        # scaler = MinMaxScaler(feature_range=(-1,1))
        # train_df[train_df.columns] = scaler.fit_transform(train_df)
        # test_df[test_df.columns] = scaler.fit_transform(test_df)
        # for col in train_df.columns:
        #     train_df.rename(columns={col:"scaled_%s"%col},inplace=True)
        # for col in test_df.columns:
        #     test_df.rename(columns={col:"scaled_%s"%col},inplace=True)


        # Fill missing values
        train_df = fill_missing_values(train_df)
        test_df  = fill_missing_values(test_df)

        # Now chunk the sets
        x_train, y_train = split_series(train_df.values,n_input_sequence,n_output_sequence,out_column_index=out_column_index)
        x_test,  y_test  = split_series(test_df.values, n_input_sequence,n_output_sequence,out_column_index=out_column_index)

        X_train = np.append(X_train,x_train,axis=0)
        Y_train = np.append(Y_train,y_train,axis=0)
        X_test  = np.append(X_test,x_test,axis=0)
        Y_test  = np.append(Y_test,y_test,axis=0)
    if (return_scaler):
        return(X_train,Y_train,X_test,Y_test,out_scaler)
    else:
        return (X_train,Y_train,X_test,Y_test)

In [20]:
Xtrain,Ytrain,Xtest,Ytest,out_scaler =  prepare_learning_sets(final,input_sequence_length_min=60,
                          output_sequence_length_min=30,test_size=0.2,
                          sampling_rate_min=5, 
                          out_column='Temperature',
                          return_scaler = True)

n_full_sequence 18
train_df                        pressure  Temperature  Humidity  windSpeed     windDir  \
Date                                                                            
2018-06-18 05:55:00  876.340446    15.911111      83.0   8.878059  107.580517   
2018-06-18 06:00:00  876.361866    15.966667      82.0   7.886871  105.868135   
2018-06-18 06:05:00  876.334354    16.188889      81.0   9.410956  111.989589   
2018-06-18 06:10:00  876.426859    16.466667      80.0  10.892408  110.356229   
2018-06-18 06:15:00  876.386192    16.466667      79.0   9.741351  110.317941   
...                         ...          ...       ...        ...         ...   
2018-06-18 17:50:00  878.915252    19.855556      70.0  13.567550  196.457311   
2018-06-18 17:55:00  878.882615    19.800000      70.0  13.876630  199.159137   
2018-06-18 18:00:00  878.934396    19.688889      70.0  14.644001  199.055965   
2018-06-18 18:05:00  878.977564    19.688889      71.0  11.478595  207.785046   
